In [ ]:
# We first import the time module to compute runtime
import time
import numpy as np
start_time = time.time()

# First, we write the function for creating box
# This is the code to make a box around a vector v = [v1 v2 ... vn]^T with parameters r = [r1 r2 ... rn]^T
def make_box(v, r):
    n = len(v)
    zeros = [0]*(n+1)
    mat = []
    # Adding the constraints for positive values
    for r1 in r:
        zeros[0] = r1
        mat.append(zeros*1)
    # Adding the constraints for negative values
    for r1 in r:
        zeros[0] = r1
        mat.append(zeros*1)   
    # print('mat zeros = ', mat) # [[0.1,0,0], [0.1,0,0], [0.1,0,0], [0.1,0,0]]
    
    for i in range(n):
        mat[i][i+1] = 1 # [[0.1,1,0], [0.1,0,1], [0.1,0,0], [0.1,0,0]]
    # print('mat pos = ', mat)
    j = 1
    for i in range(n,2*n):
        mat[i][j] = -1 # # [[0.1,1,0], [0.1,0,1], [0.1,-1,0], [0.1,0,-1]]
        j += 1
    # print('mat for constraints = ', mat)
    Box = Polyhedron(ieqs=mat, backend='ppl', base_ring=QQ)
    Box_vertices = np.array(Box.vertices_list())
    return Box_vertices

# Minkowski sum function
def minkowski_pol(V1, V2):
    V3 = []
    for v1 in V1:
        for v2 in V2:
            n = v1.size
            v = np.zeros(n)
            for i in range(n):
                v[i] = v1[i] + v2[i]
            V3.append(v)
    # print(V3)
    V4 = np.array(V3)
    return V4

def convex_hull(V1, V2):
    V2 = np.append(V1,V2)
    return V2

def linear_trans(M, V):
    # V1 = P1.vertices_list()
    # V2 = P2.vertices_list()
    V1 = []
    for v in V:
        x =  np.matmul(M,v)
        x = np.array(x)
        V1.append(x)
        # print(V1)
    V2 = np.array(V1)
    return V2

# Select the file to take input for algorithm 1
v = input("Enter the example number to run algorithm 1: ")
exp_num = v
# Evaluate the inputs
if (int(v)==1):
    input_file = open(r"inputs_new/ex1_dummy.txt")
elif (int(v)==2):
    input_file = open(r"inputs_new/ex2_balance_system.txt")
elif (int(v)==3):
    input_file = open(r"inputs_new/ex3_3deg_quadcopter.txt")
elif (int(v)==4):
    input_file = open(r"inputs_new/ex4_6deg_quadcopter.txt")

# Execute the input file to get A, B, K, X0, U0
exec(input_file.read())
input_file.close()


# Create the box
v = [0]*n
r = [ep]*n
Box = make_box(v,r)

# Opening a file to write the outputs
exp_string = "outputs/alg1_new/ex" + exp_num + ".csv"
f = open(exp_string, "a")
f.write('\nNew experiment:\n')


f.write('Iteration, Num_Ver(Uc), Num_Ver(Xc), Runtime\n')
# Running algorithm 1
Xc = X0
Uc = U0
iter = 50
for i in range(iter):
    try:
        # print('iteration = ', i+1, file=f)
        AXc = linear_trans(A, Xc)
        # print("AXc = ", AXc, file=f)
        BUc = linear_trans(B, Uc)
        # print("BUc = ", BUc, file=f)
        Xc = minkowski_pol(AXc,BUc)
        # print('Number_of_Cons(Xc) = ', len(list(Xc.Hrepresentation())), file=f)
        # print('Box = ', Box)
        # Xp = minkowski_pol(Xc, Box)
        # print('Number_of_Cons(Xp) = ', len(list(Xp.Hrepresentation())), file=f)
        Uc = linear_trans(K, Xc)
        # print('Number_of_Cons(Uc) = ', len(list(Uc.Hrepresentation())), file=f)
        # print('Vol(Xc) = ', float(Xc.volume()),  file=f)
        # print('Vol(Xp) = ', float(Xp.volume()),  file=f)
        # print('Vol(Uc) = ', float(Uc.volume()),  file=f)
        curr_time = (time.time()-start_time)*10**3
        msg = str((i+1, len(Uc), len(Xc), curr_time))
        msg = msg[1:-1]
        print(msg)
        f.write(msg+'\n')
    except KeyboardInterrupt:
        break

# Save the runtime
end_time = time.time()
total_time_in_ms = (end_time-start_time)*10**3

# msg = str((iter, len(list(Xc.Hrepresentation())), len(list(Xc.vertices())), len(list(Xc.vertices())[0]), total_time_in_ms, error_msg))
# msg = msg[1:-1]
# f.write(msg)

print("\nRuntime: ", total_time_in_ms, file = f)

# Close the file
f.close()

# Saving the plot
try:
    fig = Xc.plot()
    fig_string = "outputs/alg1/ex" + exp_num + ".png"
    fig.save(fig_string)
except:
    pass

# Plotting in notebook
Xc
